In [1]:
!pip install -Uqqq pip
!pip install -qqq bitsandbytes==0.39.0
!pip install -qqq torch==2.0.1
!pip install -qqq -U git+https://github.com/huggingface/transformers.git@e03a9cc
!pip install -qqq -U git+https://github.com/huggingface/peft.git@42a184f
!pip install -qqq -U git+https://github.com/huggingface/accelerate.git@c9fbb71
!pip install -qqq datasets==2.12.0
!pip install -qqq loralib==0.1.1
!pip install -qqq einops==0.6.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 99.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 123.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [2]:
import json
import os
from pprint import pprint
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('8013'), PosixPath('//172.28.0.1'), PosixPath('http')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

In [3]:
notebook_login()

# LOAD LLAMA MODEL & TOKENIZER

In [4]:
MODEL_NAME = "meta-llama/Llama-2-7b-chat-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [5]:
def print_trainable_parameters(model):
  """
  Prints the number of trainable parameters in the model.
  """
  trainable_params = 0
  all_param = 0
  for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
      trainable_params += param.numel()
  print(
      f"trainable params: {trainable_params} || all params: {all_param} || trainables%: {100 * trainable_params / all_param}"
  )

In [6]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [7]:
config = LoraConfig(
    r=16, # dimension of the updated matrices
    lora_alpha=32, # parameter for scaling
    target_modules=[
    "q_proj",
    "up_proj",
    "o_proj",
    "k_proj",
    "down_proj",
    "gate_proj",
    "v_proj"],
    lora_dropout=0.1, # dropout probability for layers
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)

In [8]:
print_trainable_parameters(model)

trainable params: 39976960 || all params: 3540389888 || trainables%: 1.1291682911958425


# Test original model (Der Teil funktioniert manchmal nicht. Erstmal ignorieren)

In [ ]:
#prompt = """
#<human>: midjourney prompt for a girl sit on the mountain
#<assistant>:
#""".strip()

In [ ]:
#generation_config = model.generation_config
#generation_config.max_new_tokens = 200
#generation_config.temperature = 0.7
#generation_config.top_p = 0.7
#generation_config.num_return_sequences = 1
#generation_config.pad_token_id = tokenizer.eos_token_id
#generation_config.eos_token_id = tokenizer.eos_token_id

In [ ]:
#%%time
#device = "cuda:0"

#encoding = tokenizer(prompt, return_tensors="pt").to(device)
#with torch.inference_mode():
#  outputs = model.generate(
#      input_ids = encoding.input_ids,
#      attention_mask = encoding.attention_mask,
#      generation_config = generation_config
#  )

#print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<human>: midjourney prompt for a girl sit on the mountain
<assistant>: What do you want to do on the mountain?
<human>: I want to take a break and enjoy the view.
<assistant>: Alright, take your time.
<human>: 
CPU times: user 13.4 s, sys: 224 ms, total: 13.7 s
Wall time: 16.9 s


# Prep dataset

In [9]:
data = load_dataset("csv", data_files="csv_dateiKombV2.csv")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-5a62b131c08f425c/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
data

DatasetDict({
    train: Dataset({
        features: ['User', 'Prompt'],
        num_rows: 291
    })
})

In [11]:
data["train"][0]

{'User': '"DescriptionLongMarketplaces für #### 2x Bewässerungskugel #### Lieferumfang: 2x Bewässerungskugel in Farbe nach Wahl #### Bewässerungskugeln 2er Set im Detail: Durchmesser Kugel: ca. 9 cm / Gesamtlänge: ca. 30 cm / Stiellänge: ca. 21 cm / Füllmenge je: ca. 350 ml / Material: Glas / Gewicht je: ca. 150 g / Verschiedene Farben / Durstkugeln – Einfache Handhabung: Mit Wasser befüllen und mit Spitze nach unten fest in den Blumentopf stecken / Spitze sollte sich in Wurzelnähe befinden und mit Erde gefüllt sein, um nicht auszulaufen / Funktionsweise: Erde lässt nur so viel Wasser durch, wie die Pflanze benötigt / Dauer der Bewässerung ist abhängig von Zimmertemperatur, Pflanzenart und -größe / Achtung: Bei komplett trockener Erde werden die Kugeln schneller leer #### Für Blumentöpfe / Smarte Gießhilfe / Dosierte Wasserabgabe / Aus Glas / Für den Urlaub"',
 'Prompt': '"Deine Vorfreude auf den baldigen Urlaub wäre größer, wenn Du wüsstest, dass es auch Deinen Topfpflanzen in dieser 

In [12]:
def generate_prompt(data_point):
  return f"""
<human>: {data_point["User"]}
<assistant>: {data_point["Prompt"]}
""".strip()

def generate_and_tokenize_prompt(data_point):
  full_prompt = generate_prompt(data_point)
  tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
  return tokenized_full_prompt

In [13]:
data = data["train"].shuffle().map(generate_and_tokenize_prompt)

Map:   0%|          | 0/291 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [14]:
data

Dataset({
    features: ['User', 'Prompt', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 291
})

# Finetune the model

In [15]:
training_args = transformers.TrainingArguments(
      per_device_train_batch_size=1,
      gradient_accumulation_steps=4,
      num_train_epochs=1,
      learning_rate=2e-4,
      fp16=True,
      save_total_limit=3,
      logging_steps=1,
      output_dir="experiments",
      optim="paged_adamw_8bit",
      lr_scheduler_type="cosine",
      warmup_ratio=0.05,
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,2.175500
2,2.146300
3,2.128600
4,1.986600
5,1.746000
6,1.750100
7,1.751100
8,1.610500
9,1.532700
10,1.388200


TrainOutput(global_step=72, training_loss=1.0678594071004126, metrics={'train_runtime': 1180.3892, 'train_samples_per_second': 0.247, 'train_steps_per_second': 0.061, 'total_flos': 4075409961050112.0, 'train_loss': 1.0678594071004126, 'epoch': 0.99})

# Save trained model

In [16]:
model.save_pretrained("trained-model")

In [18]:
PEFT_MODEL = "der0/test-falcon-7b"

model.push_to_hub(
    PEFT_MODEL, use_auth_token=True
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:282: UserWarning: About to update multiple times the same file in the same commit: 'adapter_model.bin'. This can cause undesired inconsistencies in your repo.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:282: UserWarning: About to update multiple times the same file in the same commit: 'adapter_config.json'. This can cause undesired inconsistencies in your repo.
  warnings.warn(


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.bin:   0%|          | 0.00/160M [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/160M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/der0/test-falcon-7b/commit/6cc25407e01ee82add961c16a83888746decc4fc', commit_message='Upload model', commit_description='', oid='6cc25407e01ee82add961c16a83888746decc4fc', pr_url=None, pr_revision=None, pr_num=None)

In [19]:
config = PeftConfig.from_pretrained("der0/test-falcon-7b")

adapter_config.json:   0%|          | 0.00/504 [00:00<?, ?B/s]

In [20]:
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    use_auth_token=True
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [21]:
tokenizer=AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, PEFT_MODEL)

adapter_model.bin:   0%|          | 0.00/160M [00:00<?, ?B/s]

# Run the finetuned model

In [22]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [23]:
%%time
device = "cuda:0"

prompt = """
DescriptionLongMarketplaces für #### 2x Bewässerungskugel #### Lieferumfang: 2x Bewässerungskugel in Farbe nach Wahl #### Bewässerungskugeln 2er Set im Detail: Durchmesser Kugel: ca. 9 cm / Gesamtlänge: ca. 30 cm / Stiellänge: ca. 21 cm / Füllmenge je: ca. 350 ml / Material: Glas / Gewicht je: ca. 150 g / Verschiedene Farben / Durstkugeln – Einfache Handhabung: Mit Wasser befüllen und mit Spitze nach unten fest in den Blumentopf stecken / Spitze sollte sich in Wurzelnähe befinden und mit Erde gefüllt sein, um nicht auszulaufen / Funktionsweise: Erde lässt nur so viel Wasser durch, wie die Pflanze benötigt / Dauer der Bewässerung ist abhängig von Zimmertemperatur, Pflanzenart und -größe / Achtung: Bei komplett trockener Erde werden die Kugeln schneller leer #### Für Blumentöpfe / Smarte Gießhilfe / Dosierte Wasserabgabe / Aus Glas / Für den Urlaub
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

DescriptionLongMarketplaces für #### 2x Bewässerungskugel #### Lieferumfang: 2x Bewässerungskugel in Farbe nach Wahl #### Bewässerungskugeln 2er Set im Detail: Durchmesser Kugel: ca. 9 cm / Gesamtlänge: ca. 30 cm / Stiellänge: ca. 21 cm / Füllmenge je: ca. 350 ml / Material: Glas / Gewicht je: ca. 150 g / Verschiedene Farben / Durstkugeln – Einfache Handhabung: Mit Wasser befüllen und mit Spitze nach unten fest in den Blumentopf stecken / Spitze sollte sich in Wurzelnähe befinden und mit Erde gefüllt sein, um nicht auszulaufen / Funktionsweise: Erde lässt nur so viel Wasser durch, wie die Pflanze benötigt / Dauer der Bewässerung ist abhängig von Zimmertemperatur, Pflanzenart und -größe / Achtung: Bei komplett trockener Erde werden die Kugeln schneller leer #### Für Blumentöpfe / Smarte Gießhilfe / Dosierte Wasserabgabe / Aus Glas / Für den Urlaub"

<p>Die Bewässerungskugeln sind die ideale Gießhilfe für Blumen, Balkonpflanzen und Gartenbeete. Die Pflanzenbehälter sind aus robustem Glas